## RAG Application using SAP HANA Cloud and OpenAI
Objectvie is to:
1. Create a table with a vector column, load the table with data
2. Call LLM with prompt and get an answer based on query
3. Fun with prompt engineering and the LLM chatbot!

Install hanal_ml python library to be able to connect to SAP HANA Cloud instance

In [ ]:
%pip install hana_ml

Connect with SAP HANA Cloud Instance and do a small test to print version and current schema. 
Address can be found from SAP HANA Cloud Central -> All Instances -> Select SAP HANA Cloud Instance name -> Connections tab -> SQL Endpoint
Port can be found from SAP HANA Cloud Central -> All Instances -> Select SAP HANA Cloud Instance name -> Connections tab -> Port
User is the SAP HANA Cloud user, if newly created then it will be DBADMIN
Password is given at the time of database creation

Below is the expected output:
4.00.000.00.1712673164 (fa/CE2024.2)
DBADMIN

In [48]:
from hana_ml import ConnectionContext

cc= ConnectionContext(
    address='<*** tenant ***>.hana.trial-us10.hanacloud.ondemand.com',
    port='443',
    user='DBADMIN',
    password='*** password ***',
    encrypt=True
    )
print(cc.hana_version())
print(cc.get_current_schema())

4.00.000.00.1712673164 (fa/CE2024.2)
DBADMIN


Now create text embeddings for each of the row and update the row with text embeddings, also known as vectors. 
Below code defines a function that calls OpenAI API and returns back text embeddings or vector representation of the given string 

In [49]:
import requests # type: ignore
import json
def get_text_embedding(text):
  url = "https://api.openai.com/v1/embeddings"
  headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer {}".format("*** OpenAI API Key ***")
  }
  data = {
    "input": text,
    "model": "text-embedding-3-small"
  }
  response = requests.post(url, headers=headers, data=json.dumps(data))
  if response.status_code == 200:
    return response.json()["data"][0]["embedding"]
  else:
    raise Exception("Error:", response.status_code, response.text)

Below code defines a function that performs an UPDATE on each of the row passed as parameter and sets the value for VECTOR_STR as the text embedding returned back from OpenAI

In [50]:
def process_row(row):
    # Perform operations on each row
    # Uncomment below to see how embeddings look like - this will slow down the proecessing since it will print the vectors for each and every row
    #print(row['CATEGORY_NAME'],get_text_embedding(row['TITLE']))
    cursor1 = cc.connection.cursor()
    sql1 = """UPDATE PRODUCTS SET VECTOR_STR = '{}' WHERE asin = '{}'""".format(get_text_embedding(row['TITLE']),row['ASIN'])

    cursor1.execute(sql1)

The below code connects to SAP HANA Cloud database and updates text embeddings for each of the row in column VECTOR_STR. It creates text embeddings using model text-embedding-3-small from GPT-3 by calling it's API. Note - the original dataset has approx 1.4M records but this example has limited text embeddings only to 5000 records since the process is very slow and even for 5000 records it can take around half an hour to complete.

In [ ]:
cursor = cc.connection.cursor()
sql = '''SELECT * FROM PRODUCTS LIMIT 5000;'''
cursor.execute(sql)
hdf = cc.sql(sql)
df_context = hdf.head(5000).collect()
df_context.apply(process_row, axis=1)
#print(df_context)
cursor.close()

Uncomment below to see how text embedding for "Gift Supplies" look like

In [21]:
#queryStr = "Gift Supplies"
#queryEmbedding = get_text_embedding(queryStr)
#print(queryEmbedding)

Below function does the vector search for PRODUCTS table based on similarity metric passed. The two possible options for metric are COSINE_SIMILARITY and L2DISTANCE. The function returns back the matching titles where k is the number of rows and query is the string to be searched for

In [51]:
def run_vector_search(query: str, metric="COSINE_SIMILARITY", k=4):
    if metric == 'L2DISTANCE':
        sort = 'ASC'
    else:
        sort = 'DESC'
    query_vector = get_text_embedding(query)
    sql = '''SELECT TOP {k} "ASIN", "TITLE" 
        FROM "PRODUCTS"
        ORDER BY "{metric}"("VECTOR", TO_REAL_VECTOR('{qv}')) {sort}'''.format(k=k, metric=metric, qv=query_vector, sort=sort)
    hdf = cc.sql(sql)
    df_context = hdf.head(k).collect()
    return df_context

Below code initiates GPT-4 model for LLM

In [53]:
from langchain_openai import ChatOpenAI # type: ignore
model = ChatOpenAI(model="gpt-4", api_key="*** OpenAI API Key ***")

Below function encapsulates LLM query. Note the various steps of creating prompt template and how parameters are passed

In [103]:
def llm_query(query: str):
    from langchain_core.output_parsers import StrOutputParser # type: ignore
    from langchain_core.prompts import ChatPromptTemplate # type: ignore
    
    # Toggle and uncomment below prompt templates to execute use cases for with and without role
    
    # Use Case 1 - without LLM role assignment
    #promptTemplate = ChatPromptTemplate.from_template('''Context:{context} Question:{query}''')
    
    # Use Case 2 - with LLM role assignment as salesman
    promptTemplate = ChatPromptTemplate.from_template('''You are a salesman who promotes and sells online products. When pitching for the product, clearly mention the brand names, 
                                            it's multiple uses, benefits and cost - include a 10% discount if buying today. Use the following pieces of context to answer 
                                            the question at the end. Context:{context} Question:{query}''')
    df_vectorcontext = run_vector_search(query)
    context =  ' '.join(df_vectorcontext['TITLE'].astype('string'))
    #print(context)
    prompt = promptTemplate.format(query=query, context=context)
    response = model.invoke(prompt).content
    return (response)


Test different queries with below statement and note the responses

In [97]:
import textwrap
iWidth = 80
def print_responses(query: str):
    print("Results of query '" + query + "' for vector search:\n")
    df = run_vector_search(query=query)
    print(df.head())
    print("\nResults of query '" + query + "' from LLM:\n")
    print(llm_query(query=query))

In [104]:
#Query 1 - plain keyword search
print_responses(query="electronics")

Results of query 'electronics' for vector search:

         ASIN                                              TITLE
0  B09V3DH4G9  DC 12V Electronic DC Buzzer 5v Alarm Active Pi...
1  B08G518BHN  DC 3-24V Active Piezo Buzzer 85 db with Indust...
2  B01MYPZZ19  IQ Toys-AB A+B Test Mind Game Brain Teaser Wir...
3  B0C7CNKGTT  cssopenss 6 Pieces Pocket Screwdriver Mini Top...

Results of query 'electronics' from LLM:

Salesman: Ladies and Gentlemen, I bring you a collection of high-quality, versatile electronic products that will cater to all your needs. First, let's talk about the DC 12V Electronic DC Buzzer. This fantastic device, housed in durable ABS, emits a clear 85dB beep sound, perfect for alarms or alert systems. Wired and ready to connect, it's available in a pack of 26 for just $50. Purchase today, and you'll receive a 10% discount, making it only $45.

For those in the industrial sector, the DC 3-24V Active Piezo Buzzer is a must-have. With 85 dB of industrial audio inside and

In [105]:
#Query 2 - phrase search
print_responses(query="electronics entertainment devices")

Results of query 'electronics entertainment devices' for vector search:

         ASIN                                              TITLE
0  B09V3DH4G9  DC 12V Electronic DC Buzzer 5v Alarm Active Pi...
1  B0057PLN9K  New Remote Control VR15-0980-0306-0302 Fit for...
2  B091699Z3W  Kasa Smart Plug Ultra Mini 15A, Smart Home Wi-...
3  B01BGUX78E  Ceptics 2 USB Israel Travel Adapter 4 in 1 Pow...

Results of query 'electronics entertainment devices' from LLM:

Salesman: Greetings! I have some exciting products in the electronics and entertainment devices sector that I'm sure you'll find interesting. 

Firstly, we have the DC 12V Electronic DC Buzzer. This high-quality buzzer comes with a 5v active piezo beep sound, ABS housing, and an 85dB wired connector. Its versatility means it can be used in various applications, such as alarms, timers, or even as a fun DIY electronic project component. Each pack containing 26 pieces costs just $20, and if you purchase it today, you can avail a 10% d

In [106]:
#Query 3 - Popularity search
print_responses(query="What are top 10 electronics entertainment devices?")

Results of query 'What are top 10 electronics entertainment devices?' for vector search:

         ASIN                                              TITLE
0  B08798HG7F  Stylus Pens for Touch Screens, StylusHome 10 P...
1  B0BQD2TZSL                  Men's Top Ten Hi Basketball Shoes
2  B07C87CX9T  10 Plagues Passover Finger Puppets Vinyl Gift ...
3  B07L5RVRR2  T Shirt Boys Creeper Inside Black Short Sleeve...

Results of query 'What are top 10 electronics entertainment devices?' from LLM:

Salesman: Ladies and Gentlemen, I am here to introduce you to a number of fantastic products that will cater to your electronic and entertainment needs today. 

Firstly, let me present the StylusHome 10 Pack Mesh Fiber Tip Stylus Pens. These are ideal for touch screen devices, suitable for iPad, iPhone, Samsung tablets, and all precision capacitive universal touch screen devices. They provide precise control for selecting icons, writing, drawing and gaming. You can own this pack of 10 pens today fo

In [107]:
#Query 4 - Question that needs semantic knowledge
print_responses(query="Which items do you recommend for my dental needs?")

Results of query 'Which items do you recommend for my dental needs?' for vector search:

         ASIN                                              TITLE
0  B07PNPSZ18  Easyinsmile Dental Tofflemire Matrices Bands S...
1  B0BX9KQWDK  315 Count Floss Threaders for Braces, Dental F...
2  B07NPBY16M  RADIUS Toothbrush Source Floss Brush BPA Free ...
3  B00JVF4UZ6  Tenders Pre-Toothbrush, 12 Individually Packag...

Results of query 'Which items do you recommend for my dental needs?' from LLM:

Salesman: Absolutely, I have some fantastic recommendations for your dental needs that not only contribute to healthy teeth and gums but also make your daily oral hygiene routine more efficient. 

First, we have the Easyinsmile Dental Tofflemire Matrices Bands. These are made of stainless steel and are excellent orthodontic supplies. With a thickness of 0.04 mm, you get 144 pieces in one pack. They greatly assist in achieving optimal contour and tooth separation during dental procedures. They are a m